In [697]:
import requests
import json
import pandas as pd
from geopy import Nominatim
import plotly_express as px

def get_keys(path):
    #  retrieve your key/token from json file
    with open(path) as f:
        return json.load(f)
    
your_path = "/Users/aaronlee/Desktop/toa_login.json"  # Make a json file that stores your key
keys = get_keys(your_path)

api_key = keys['key']

# Map all the teams

I will start by doing a simple team query.  
Let's get all of the active teams for this year, and maybe plot their location or at least grab their locations using Nomimatim.

We can do a popup chloropleth that shows stats for each state.

Once we do the big query, we can pickle or write to csv.

### Get Active teams
This step will just get teams playing this year.

In [698]:
def get_active_teams(year):
    '''fetch a list of active teams for current year (or other)'''
    
    url = "https://theorangealliance.org/api/team" # list of team dicts

    
    headers = {'Content-Type': 'application/json', 
          'X-TOA-Key': api_key, 
          'X-Application-Origin': 'toa_map',
         }

    params = {#'country':'Canada',
              #'team_key': team_list,
              'last_active': year,
             }
    r = requests.get(url, headers=headers, params=params)
    if r.status_code != 200:
        print("Failed, status code:", r.status_code)
        return None
        
    my_json = r.json()
    
    active_teams = pd.DataFrame(my_json)
    return active_teams

def query_team(team):
    url = "https://theorangealliance.org/api/team/{}".format(team)  # list of team dicts

    
    headers = {'Content-Type': 'application/json', 
          'X-TOA-Key': api_key, 
          'X-Application-Origin': 'toa_map',
         }

    params = {#'country':'Canada',
              #'team_key': team,
             }
    r = requests.get(url, headers=headers, params=params)
    if r.status_code != 200:
        print("Failed, status code:", r.status_code)
        return None
        
    my_json = r.json()
    
    df = pd.DataFrame(my_json)
    return df

active_teams = get_active_teams(2021)  # 2021 means 2020-2021
#query_team(3507)


In [699]:
active_teams.head()

,team_key,region_key,league_key,team_number,team_name_short,team_name_long,robot_name,last_active,city,state_prov,zip_code,country,rookie_year,website
0,1,MA,None,1,Team Unlimited,PTC & FTC1 Team Unlimited 4-H Club,None,2021,Sharon,MA,02067,USA,2007,http://www.ftc0001.org/
1,7,MD,None,7,"Tactical Sheep, Team 7",TechBrick Education/DoD STEM&TechBrick Education,None,2021,Aberdeen Proving Ground,MD,21005,USA,2007,http://www.techbrick.com
2,18,NY,None,18,Techno Chix,Girl Scouts Heart of the Hudson,None,2021,Pleasantville,NY,10570,USA,2007,http://www.thetechnoteams.com
3,22,GA,None,22,100 Scholars,100 Black Men of Atlanta/Georgia Power/Johnson...,None,2021,Atlanta,GA,30303,USA,2007,None
4,25,CA,None,25,Rock N' Roll Robots,Girl Scouts of Greater Los Angeles & Girl Scou...,None,2021,Pasadena,CA,91006,USA,2007,http://www.rocknrollrobots25.com


### Let's start by just using a state stats visualization in plotly.  
We can move it to Flask later

In [700]:
plot_active = active_teams.groupby(['state_prov']).count()[['team_key', 'zip_code', 'team_name_short']].reset_index()
plot_active['State'] = plot_active['state_prov']
plot_active['total_teams'] = plot_active['team_key']

In [701]:

fig = px.choropleth(data_frame=plot_active,
                    locations='State', 
                    locationmode="USA-states", 
                    color='total_teams', 
                    hover_name='State',
                    hover_data={'total_teams':True,
                                'State':False,},
                    color_continuous_scale='Plotly3',
                    scope="usa")

fig.update_layout(
    coloraxis_colorbar=dict(
        title="Active Teams",
    )
)

fig.show()

That was a good start.  I have a chlorpleth colored map showing number of teams in state.

### Would like to see a zipcode map, but likely will not work

In [717]:
zip_codes = pd.read_csv('us-zip-code-latitude-and-longitude.csv', delimiter=';')
zip_codes.head()
zip_codes['Zip'] = zip_codes['Zip'].apply(lambda x: '{:05}'.format(x))
zip_codes.sort_values(by=['Zip'])
#active_teams['zip_code'] = active_teams['zip_code'].astype(int)
zip_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43191 entries, 0 to 43190
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Zip                         43191 non-null  object 
 1   City                        43191 non-null  object 
 2   State                       43191 non-null  object 
 3   Latitude                    43191 non-null  float64
 4   Longitude                   43191 non-null  float64
 5   Timezone                    43191 non-null  int64  
 6   Daylight savings time flag  43191 non-null  int64  
 7   geopoint                    43191 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 2.6+ MB


In [718]:
zip_codes
zip_codes_crunch = zip_codes.groupby(['State', 'City']).mean()
zip_codes

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,55795,Willow River,MN,46.317812,-92.843150,-6,1,"46.317812,-92.84315"
1,45388,Yorkshire,OH,40.328535,-84.479380,-5,1,"40.328535,-84.47938"
2,39483,Foxworth,MS,31.218509,-89.907610,-6,1,"31.218509,-89.90761"
3,31503,Waycross,GA,31.205194,-82.375340,-5,1,"31.205194,-82.37534"
4,45833,Delphos,OH,40.841409,-84.341780,-5,1,"40.841409,-84.34178"
...,...,...,...,...,...,...,...,...
43186,25612,Chauncey,WV,37.766620,-81.987550,-5,1,"37.76662,-81.98755"
43187,59444,Galata,MT,48.635319,-111.240830,-7,1,"48.635319,-111.24083"
43188,60532,Lisle,IL,41.789079,-88.085360,-6,1,"41.789079,-88.08536"
43189,61638,Peoria,IL,40.796887,-89.611146,-6,1,"40.796887,-89.611146"


In [719]:
#active_teams.merge(how='left', right=zip_codes, left_on='zip_code', right_on='Zip', validate='many_to_many').info()
# need something more like a lookup cest la vie
active_usa = active_teams[active_teams['country']=="USA"]
active_usa = active_usa.groupby(['zip_code'])['team_key'].apply(list).reset_index()

active_usa = active_teams[active_teams['country']=="USA"]
team_names = active_usa.groupby(['zip_code'])['team_name_short'].apply(list).reset_index()['team_name_short']
team_numbers = active_usa.groupby(['zip_code'])['team_key'].apply(list).reset_index()['team_key']

active_usa = active_usa.groupby(['zip_code']).mean().reset_index()

active_usa['team_names'] = team_names
active_usa['team_numbers'] = team_numbers

active_usa = active_usa.merge(how='left', right=zip_codes, left_on='zip_code', right_on='Zip')
active_usa['Teams'] = active_usa['team_numbers'].apply(lambda x: len(x))
active_usa['header'] = active_usa['zip_code'].apply(lambda x: 'Zip Code: {}'.format(x))
active_usa['team_full'] = active_usa.apply(lambda x: list(zip(x.team_numbers, x.team_names)), axis=1)
active_usa.head()
active_usa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2337 entries, 0 to 2336
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   zip_code                    2337 non-null   object 
 1   team_number                 2337 non-null   float64
 2   rookie_year                 2337 non-null   float64
 3   team_names                  2337 non-null   object 
 4   team_numbers                2337 non-null   object 
 5   Zip                         2286 non-null   object 
 6   City                        2286 non-null   object 
 7   State                       2286 non-null   object 
 8   Latitude                    2286 non-null   float64
 9   Longitude                   2286 non-null   float64
 10  Timezone                    2286 non-null   float64
 11  Daylight savings time flag  2286 non-null   float64
 12  geopoint                    2286 non-null   object 
 13  Teams                       2337 

### Let's try it with city/state instead of zip

In [720]:
#active_teams.merge(how='left', right=zip_codes, left_on='zip_code', right_on='Zip', validate='many_to_many').info()
# need something more like a lookup cest la vie
active_usa = active_teams[active_teams['country']=="USA"]
active_usa = active_usa.groupby(['state_prov', 'city'])['team_key'].apply(list).reset_index()

active_usa = active_teams[active_teams['country']=="USA"]
team_names = active_usa.groupby(['state_prov','city'])['team_name_short'].apply(list).reset_index()['team_name_short']
team_numbers = active_usa.groupby(['state_prov','city'])['team_key'].apply(list).reset_index()['team_key']

active_usa = active_usa.groupby(['state_prov','city']).mean().reset_index()

active_usa['team_names'] = team_names
active_usa['team_numbers'] = team_numbers

active_usa = active_usa.merge(how='left', right=zip_codes_crunch, left_on=['state_prov', 'city'], right_on=['State', 'City'])
active_usa['Teams'] = active_usa['team_numbers'].apply(lambda x: len(x))
active_usa['header'] = active_usa.apply(lambda x: x.city + ',' + x.state_prov, axis=1)
active_usa['team_full'] = active_usa.apply(lambda x: list(zip(x.team_numbers, x.team_names)), axis=1)
active_usa.head()
active_usa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1746 entries, 0 to 1745
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   state_prov                  1746 non-null   object 
 1   city                        1746 non-null   object 
 2   team_number                 1746 non-null   float64
 3   rookie_year                 1746 non-null   float64
 4   team_names                  1746 non-null   object 
 5   team_numbers                1746 non-null   object 
 6   Latitude                    1647 non-null   float64
 7   Longitude                   1647 non-null   float64
 8   Timezone                    1647 non-null   float64
 9   Daylight savings time flag  1647 non-null   float64
 10  Teams                       1746 non-null   int64  
 11  header                      1746 non-null   object 
 12  team_full                   1746 non-null   object 
dtypes: float64(6), int64(1), object(6

In [721]:
def ready_team_names(team_list):
    my_str = ''
    for team in team_list:
        my_str += ' '.join(team) + '<br>'
    return my_str
    

active_usa['team_full'] = active_usa.team_full.apply(ready_team_names)
active_usa.sort_values(by=['Teams'], ascending=True)

,state_prov,city,team_number,rookie_year,team_names,team_numbers,Latitude,Longitude,Timezone,Daylight savings time flag,Teams,header,team_full
1745,WY,Worland,18239.000000,2020.000000,[Washakie Robotics],[18239],43.993124,-107.973650,-7.0,1.0,1,"Worland,WY",18239 Washakie Robotics<br>
1011,NE,South Sioux City,7924.000000,2013.000000,[Red Toques],[7924],42.467095,-96.418700,-6.0,1.0,1,"South Sioux City,NE",7924 Red Toques<br>
1009,NE,Norfolk,18139.000000,2019.000000,[Rebel Robotics FTC],[18139],41.971343,-97.509379,-6.0,1.0,1,"Norfolk,NE",18139 Rebel Robotics FTC<br>
1008,NE,Lincoln,8824.000000,2014.000000,[RoboStorm],[8824],40.817815,-96.691563,-6.0,1.0,1,"Lincoln,NE",8824 RoboStorm<br>
1006,NE,Humboldt,11968.000000,2016.000000,[Titan Robotics],[11968],40.172733,-95.942600,-6.0,1.0,1,"Humboldt,NE",11968 Titan Robotics<br>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,TX,Austin,12351.823529,2015.941176,"[Iron Eagles Optimus, Pi-Rho Maniacs, ViperBot...","[3708, 3781, 4545, 4546, 5628, 6209, 6210, 629...",30.319931,-97.758070,-6.0,1.0,34,"Austin,TX",3708 Iron Eagles Optimus<br>3781 Pi-Rho Maniac...
1507,TX,Houston,15889.589744,2018.333333,"[Beta Testers, Momentum, Lady Gears, KI Falcon...","[9191, 10055, 10064, 10963, 10982, 11227, 1152...",29.799726,-95.419938,-6.0,1.0,39,"Houston,TX",9191 Beta Testers<br>10055 Momentum<br>10064 L...
82,CA,Fremont,15053.000000,2017.775000,"[SpectreBots, Ink and Metal, RoboAvatars, Hype...","[596, 5773, 7303, 9614, 11039, 11099, 11689, 1...",37.571246,-121.972758,-8.0,1.0,40,"Fremont,CA",596 SpectreBots<br>5773 Ink and Metal<br>7303 ...
695,MI,Detroit,15041.292683,2017.756098,"[Aztechs, TechnoLions, SAY Play Warriors 2, Ro...","[10210, 10280, 11438, 12467, 13995, 13996, 139...",42.303908,-83.127801,-5.0,1.0,41,"Detroit,MI",10210 Aztechs<br>10280 TechnoLions<br>11438 SA...


In [722]:
import plotly.express as px


fig = px.scatter_geo(active_usa, 
                     lat="Latitude", 
                     lon='Longitude',
                     color="Teams",
                     hover_name="header", 
                     hover_data=['team_full'],
                     size='Teams', 
                     #size_max=30, 
                     color_continuous_scale='Plotly3',
                     opacity=0.7,
                    )

fig.update_layout(mapbox_style="open-street-map",)

fig.update_layout(
        title_text = 'FTC Teams by Zipcode (USA only)',
        showlegend = True,
        geo = dict(
            scope = 'usa',
            landcolor = 'rgb(150, 150, 150)',
        )
    )

fig.show()

In [563]:
fig = px.density_mapbox(active_usa, 
                     lat="Latitude", 
                     lon='Longitude',
                     hover_name="header", 
                    color_continuous_scale='electric',
                        zoom=3,
                        radius=7
                    )

fig.update_layout(mapbox_style="carto-positron",)

fig.update_layout(
        title_text = 'FTC Teams by Zipcode (USA only)',
        showlegend = False,

    )

fig.show()

In [564]:
fig = px.scatter_mapbox(active_usa,
                       lat='Latitude',
                       lon='Longitude',
                        zoom=3,

                       )

fig.update_layout(mapbox_style="open-street-map",)

fig.update_geos(scope='usa')

fig.show()




### Let's make it cooler
I would like to make a map that is by zipcode, but shows the team info when you scroll over it.  
That might be tricky.  I need to make a column that is teams and team names.  

Strategy:
- assign zip to every team. (most versatile method)
- group by zip

In [363]:
headers = {'Content-Type': 'application/json', 
          'X-TOA-Key': api_key, 
          'X-Application-Origin': 'toa_map',
         }

params = {'country':'Canada',
          'team_key'
         }

# TL;DR:
# Set Content-Type to application/json
# Set X-TOA-Key to your API Key
# Set X-Application-Origin to your application name
url = "https://theorangealliance.org/api/seasons"  # list of season dicts
url = "https://theorangealliance.org/api/team"  # list of team dicts
url = "https://theorangealliance.org/api/team/size"  # number of db entries for teams
url = "https://theorangealliance.org/api/team/1"  # single team info
url = "https://theorangealliance.org/api/leagues"  # single team info
url = "https://theorangealliance.org/api/team/10101/"  # single team info
#url = "https://theorangealliance.org/api/team/9410/matches/2021"
#url = "https://theorangealliance.org/api/match/high-scores"
url = "https://theorangealliance.org/api/team/size"


r = requests.get(url, headers=headers, params=params)
print(r.status_code)

200


In [364]:
r.content

b'{"result":358}'

In [361]:
r.url
my_json = r.json()
my_json
print(type(my_json))
df = pd.DataFrame(my_json)

<class 'list'>


In [362]:
df

,team_key,region_key,league_key,team_number,team_name_short,team_name_long,robot_name,last_active,city,state_prov,zip_code,country,rookie_year,website
0,27,ON,None,27,None,None,None,0708,Toronto,ON,,Canada,2007,http://
1,62,AB,None,62,None,Hunting Hills High School,None,0708,Red Deer,AB,,Canada,2007,http://
2,104,ON,None,104,None,None,None,0708,Corbeil,ON,,Canada,2007,http://
3,106,ON,None,106,None,None,None,0708,Aurora,ON,,Canada,2007,http://
4,122,ON,None,122,None,Dr. Norman Bethune C.I. High School,None,0708,Toronto,ON,,Canada,2007,http://
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,19133,NL,None,19133,Immaculate Heart of Mary School,Immaculate Heart of Mary School,None,2021,Corner Brook,NL,A2H 1E8,Canada,2020,None
354,19138,ON,None,19138,BT Robotics1,Bishop Tonnos Secondary School,None,2021,Ancaster,ON,L9G 5E3,Canada,2020,None
355,19189,ON,None,19189,OP Robotics,Orchard Park Secondary School,None,2021,Stoney Creek,ON,L8E 4S7,Canada,2020,None
356,19205,ON,None,19205,OP Robotics2,Orchard Park Secondary School,None,2021,Stoney Creek,ON,L8E 4S7,Canada,2020,None


# get all match data

In [314]:
df.match_key

0      2021-IL-IFAMR-Q001-1-3507
1      2021-IL-IFAMR-Q002-1-3507
2      2021-IL-IFAMR-Q003-1-3507
3      2021-IL-IFAMR-Q004-1-3507
4      2021-IL-IFAMR-Q005-1-3507
5      2021-IL-IFAMR-Q006-1-3507
6     2021-IL-IFAMR1-Q001-1-3507
7     2021-IL-IFAMR1-Q002-1-3507
8     2021-IL-IFAMR1-Q003-1-3507
9     2021-IL-IFAMR1-Q004-1-3507
10    2021-IL-IFAMR1-Q005-1-3507
11    2021-IL-IFAMR1-Q006-1-3507
12    2021-IL-IFAMR2-Q001-1-3507
13    2021-IL-IFAMR2-Q002-1-3507
14    2021-IL-IFAMR2-Q003-1-3507
15    2021-IL-IFAMR2-Q004-1-3507
16    2021-IL-IFAMR2-Q005-1-3507
17    2021-IL-IFAMR2-Q006-1-3507
18    2021-IL-IFAMR3-Q001-1-3507
19    2021-IL-IFAMR3-Q002-1-3507
20    2021-IL-IFAMR3-Q003-1-3507
21    2021-IL-IFAMR3-Q004-1-3507
22    2021-IL-IFAMR3-Q005-1-3507
23    2021-IL-IFAMR3-Q006-1-3507
24     2021-IL-IFASR-Q001-1-3507
25     2021-IL-IFASR-Q002-1-3507
26     2021-IL-IFASR-Q003-1-3507
27     2021-IL-IFASR-Q004-1-3507
28     2021-IL-IFASR-Q005-1-3507
29     2021-IL-IFASR-Q006-1-3507
Name: matc

In [315]:
for i, match_key in enumerate(df.match_key):
    url = "https://theorangealliance.org/api/match/{}".format(match_key)
    r = requests.get(url, headers=headers)
    print('i: {},  Status code: {}'.format(i, r.status_code))
    my_json = r.json()
    
    if i == 0: 
        new_df = pd.DataFrame(my_json)
        continue

    new_df = new_df.append(pd.DataFrame(my_json))
    

i: 0,  Status code: 200
i: 1,  Status code: 200
i: 2,  Status code: 200
i: 3,  Status code: 200
i: 4,  Status code: 200
i: 5,  Status code: 200
i: 6,  Status code: 200
i: 7,  Status code: 200
i: 8,  Status code: 200
i: 9,  Status code: 200
i: 10,  Status code: 200
i: 11,  Status code: 200
i: 12,  Status code: 200
i: 13,  Status code: 200
i: 14,  Status code: 200
i: 15,  Status code: 200
i: 16,  Status code: 200
i: 17,  Status code: 200
i: 18,  Status code: 200
i: 19,  Status code: 200
i: 20,  Status code: 200
i: 21,  Status code: 200
i: 22,  Status code: 200
i: 23,  Status code: 200
i: 24,  Status code: 200
i: 25,  Status code: 200
i: 26,  Status code: 200
i: 27,  Status code: 200
i: 28,  Status code: 200
i: 29,  Status code: 429


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [319]:
new_df.red_score.max()

60

In [273]:
r.url
my_json = r.json()
my_json
print(type(my_json))
df = pd.DataFrame(my_json)

<class 'list'>


In [283]:
pd.options.display.max_columns = 100
df

,match_key,event_key,tournament_level,scheduled_time,match_name,play_number,field_number,prestart_time,match_start_time,prestart_count,cycle_time,red_score,blue_score,red_penalty,blue_penalty,red_auto_score,blue_auto_score,red_tele_score,blue_tele_score,red_end_score,blue_end_score,video_url,participants
0,2021-MA-MFFSR1-Q005-1-1,2021-MA-MFFSR1,1,2001-01-01T00:00:00.000Z,Quals 5,1,-1,None,None,None,None,0,-1,0,-1,0,-1,0,-1,0,-1,None,[{'match_participant_key': '2021-MA-MFFSR1-Q00...
